In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
from matplotlib.image import imsave
import os

In [5]:
import cv2
import numpy as np
import os

# Define the paths to the directories containing the images
foreground_dir = '/mnt/DATA/dronetracking/final_composition/foreground/video01/foreground'
background_dir = '/mnt/DATA/dronetracking/Anti-UAV-Tracking-V0/video02'
mask_dir = '/mnt/DATA/dronetracking/final_composition/foreground/video01/mask_original_size'
output_dir = '/mnt/DATA/dronetracking/final_composition/compostion_result/comp1_2_4'

# List all files in the directories
foreground_files = sorted(os.listdir(foreground_dir))
background_files = sorted(os.listdir(background_dir))
mask_files = sorted(os.listdir(mask_dir))

# Determine the minimum number of files among the directories
min_files = min(len(background_files), len(foreground_files), len(mask_files))

# Define the intervals explicitly (these are example ranges, you can adjust them)
intervals = [
    (0, 152),     # First interval (e.g., 0 to 49)
    (284, 359),    # Second interval (e.g., 50 to 99)
    (374, 750),  # Third interval (e.g., 100 to 149)
    (938, 1150)  # Fourth interval (e.g., 150 to min_files-1)
]

# Calculate interval indices
interval_indices = [list(range(start, end + 1)) for start, end in intervals]

# Process each image pair until you run out of background images
for i in range(min_files):
    background_file = background_files[i]
    background_path = os.path.join(background_dir, background_file)
    background = cv2.imread(background_path)

    # Initialize the combined image with the background
    combined_image = background.copy()

    # Process each interval and overlay the foregrounds
    for interval_idx, indices in enumerate(interval_indices):
        if i < len(indices):
            fg_index = indices[i]
            foreground_file = foreground_files[fg_index]
            mask_file = mask_files[fg_index]

            # Load the images
            foreground_path = os.path.join(foreground_dir, foreground_file)
            mask_path = os.path.join(mask_dir, mask_file)

            foreground = cv2.imread(foreground_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load mask as grayscale

            # Resize the foreground and mask to match the background size if necessary
            foreground_resized = cv2.resize(foreground, (background.shape[1], background.shape[0]))
            mask_resized = cv2.resize(mask, (background.shape[1], background.shape[0]))

            # Convert segmentation mask to binary
            _, mask_binary = cv2.threshold(mask_resized, 1, 255, cv2.THRESH_BINARY)

            # Invert the mask
            mask_inv = cv2.bitwise_not(mask_binary)

            # Apply the inverted mask to the background
            background_bg = cv2.bitwise_and(combined_image, combined_image, mask=mask_inv)

            # Combine the foreground with the background
            foreground_fg = cv2.bitwise_and(foreground_resized, foreground_resized, mask=mask_binary)
            combined_image = cv2.add(background_bg, foreground_fg)

            print(f"Processed {foreground_file}, {background_file}, {mask_file} in interval {interval_idx + 1}")

    # Save the final combined image
    output_path = os.path.join(output_dir, f'combined_{i:05d}.png')
    cv2.imwrite(output_path, combined_image)

    print(f"Saved final combined image to {output_path}")

print("All images processed and combined.")


Processed result_00001.png, 00001.jpg, c_0001.png in interval 1
Processed result_00285.png, 00001.jpg, c_0285.png in interval 2
Processed result_00375.png, 00001.jpg, c_0375.png in interval 3
Processed result_00939.png, 00001.jpg, c_0939.png in interval 4
Saved final combined image to /mnt/DATA/dronetracking/final_composition/compostion_result/comp1_2_4/combined_00000.png
Processed result_00002.png, 00002.jpg, c_0002.png in interval 1
Processed result_00286.png, 00002.jpg, c_0286.png in interval 2
Processed result_00376.png, 00002.jpg, c_0376.png in interval 3
Processed result_00940.png, 00002.jpg, c_0940.png in interval 4
Saved final combined image to /mnt/DATA/dronetracking/final_composition/compostion_result/comp1_2_4/combined_00001.png
Processed result_00003.png, 00003.jpg, c_0003.png in interval 1
Processed result_00287.png, 00003.jpg, c_0287.png in interval 2
Processed result_00377.png, 00003.jpg, c_0377.png in interval 3
Processed result_00941.png, 00003.jpg, c_0941.png in inter

AttributeError: 'NoneType' object has no attribute 'copy'